## For a folder where to find the train.log files

In [ ]:
import os
import re
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl

# LaTeX style setup
pgf_with_latex = {
    'pgf.texsystem': 'pdflatex',
    'text.usetex': True,
    'font.size': 10,
    'font.family': 'serif',
    'font.serif': [],
    'font.sans-serif': [],
    'font.monospace': [],
    'text.latex.preamble': r'\usepackage[utf8]{inputenc}\usepackage[T1]{fontenc}\usepackage[detect-all]{siunitx}\usepackage{amsmath}\usepackage{bm}\usepackage{color}'
}

mpl.rcParams.update(pgf_with_latex)
plt.rcParams["axes.axisbelow"] = True  # Ensure that the grid is drawn behind everything else

def parse_log_file(file_path):
    iterations = []
    rewards = []
    with open(file_path, 'r') as file:
        for line in file:
            iteration_match = re.search(r'Learning iteration (\d+)/', line)
            if iteration_match:
                iteration = int(iteration_match.group(1))
            
            reward_match = re.search(r'Mean reward: ([\d\.-]+)', line)
            if reward_match:
                reward = float(reward_match.group(1))
                iterations.append(iteration)
                rewards.append(reward)
    return iterations, rewards

def process_logs_in_directory(directory_path):
    # Define the log files of interest
    log_files = ['obs_train.log', 'rough_train.log']
    
    # Process each log file
    for log_file_name in log_files:
        log_file_path = os.path.join(directory_path, log_file_name)
        if not os.path.isfile(log_file_path):
            print(f"Log file {log_file_name} not found in {directory_path}, skipping...")
            continue

        plot_file_name = log_file_name.replace('.log', '.png')
        plot_path = os.path.join(directory_path, plot_file_name)
        
        # Extract data from log file
        iterations, rewards = parse_log_file(log_file_path)
        
        if not iterations or not rewards:
            print(f"No valid data found in {log_file_name}, skipping...")
            continue
        
        # Determine title and subtitle
        if 'obs' in log_file_name:
            title = "Obstacle Avoidance Locomotion"
        elif 'rough' in log_file_name:
            title = "General Purpose Locomotion"
        else:
            title = log_file_name.replace('.log', '')

        mean_reward = sum(rewards) / len(rewards) if rewards else 0
        best_reward = max(rewards) if rewards else 0
        best_iteration = iterations[rewards.index(best_reward)] if rewards else 0
        subtitle = (f"Mean Reward: \\textbf{{{mean_reward:.2f}}} | "
                    f"Best Reward: \\textbf{{{best_reward:.2f}}} (iter: \\textbf{{{best_iteration}}})")
        
        # Generate the plot
        plt.figure(figsize=(10, 5))
        plt.grid(True, zorder=0)  # Draw grid behind the plot
        plt.plot(iterations, rewards, zorder=1)  # Plot the line above the grid
        plt.xlabel('Iterations')
        plt.ylabel('Mean Reward')
        plt.title(f'{title}\n{subtitle}', fontsize=16)  # Increase title size only
        
        if len(iterations) > 1:
            plt.xticks(ticks=range(min(iterations), max(iterations)+1, (max(iterations) - min(iterations)) // 13))
        
        # Save the figure
        plt.savefig(plot_path)
        plt.close()
        print(f"Figure saved as {plot_path}")

def traverse_directories(parent_directory):
    # Use glob to find all subdirectories
    subdirectories = [d for d in glob.glob(os.path.join(parent_directory, '*/')) if not os.path.basename(d).startswith('.')]

    # Iterate through each subdirectory
    for subdir in subdirectories:
        # Check if the directory contains the log files
        contains_logs = any(os.path.isfile(os.path.join(subdir, log_file)) for log_file in ['obs_train.log', 'rough_train.log'])
        
        if contains_logs:
            process_logs_in_directory(subdir)
        else:
            print(f"No relevant log files found in {subdir}, skipping...")

# Example usage:
parent_folder = '/home/josep-barbera/Documents/nViNL/experiments/example_experiment_20240823_063325/obstacles'  # Change this to the path of the parent folder
traverse_directories(parent_folder)


## For the experiments folders directly with more than one run per simulation

In [19]:
import os
import re
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages

# LaTeX style setup
pgf_with_latex = {
    'pgf.texsystem': 'pdflatex',
    'text.usetex': True,
    'font.size': 10,
    'font.family': 'serif',
    'font.serif': [],
    'font.sans-serif': [],
    'font.monospace': [],
    'text.latex.preamble': r'\usepackage[utf8]{inputenc}\usepackage[T1]{fontenc}\usepackage[detect-all]{siunitx}\usepackage{bm}\usepackage{amsmath}\usepackage{color}\usepackage{amsbsy}'
}

mpl.rcParams.update(pgf_with_latex)
plt.rcParams["axes.axisbelow"] = True  # Ensure that the grid is drawn behind everything else



def parse_log_file(file_path):
    iterations = []
    rewards = []
    with open(file_path, 'r') as file:
        for line in file:
            iteration_match = re.search(r'Learning iteration (\d+)/', line)
            if iteration_match:
                iteration = int(iteration_match.group(1))
            
            reward_match = re.search(r'Mean reward: ([\d\.-]+)', line)
            if reward_match:
                reward = float(reward_match.group(1))
                iterations.append(iteration)
                rewards.append(reward)
    return iterations, rewards

def parse_experiment_log(file_path):
    times = []
    best_checkpoint = None
    with open(file_path, 'r') as file:
        for line in file:
            time_match = re.search(r'Training completed in (\d{2}:\d{2}:\d{2})', line)
            if time_match:
                print(time_match)
                time_str = time_match.group(1)
                # Convert time string (HH:MM:SS) to seconds
                h, m, s = map(int, time_str.split(':'))
                total_seconds = h * 3600 + m * 60 + s
                times.append(total_seconds)
            
            best_checkpoint_match = re.search(r'Next stage with best rough checkpoint: (\d+_\d+\.\d+)', line)
            if best_checkpoint_match:
                best_checkpoint = best_checkpoint_match.group(1)

    return times, best_checkpoint

def get_studied_parameter_from_folder_name(folder_name):
    print(folder_name)
    # Adjust the regex to match the parameter name after the timestamp
    match = re.search(r'^\./\d{8}_\d{6}_(\w+)_\d*\.?\d+/?$', folder_name)
    if match:
        return match.group(1)  # return the parameter name
    else:
        return None

def extract_ppo_parameters(file_path):
    parameters = []
    inside_ppo_section = False
    with open(file_path, 'r') as file:
        for line in file:
            if "##### PPO PARAMETERS ######" in line:
                inside_ppo_section = True
            elif "################################################################################" in line:
                if inside_ppo_section:
                    break  # Exit after the PPO section ends
            elif inside_ppo_section:
                # Adjust regex to capture only lines with parameter name and value
                param_match = re.search(r'^\s*(\w+):\s+([\d\.\-e]+)\s*$', line)
                if param_match:
                    parameters.append(f"{param_match.group(1)}: {param_match.group(2)}")
    return parameters

def process_run_directories(run_directories, log_file_name):
    all_iterations = {}
    all_run_rewards = []

    for run_dir in run_directories:
        log_file_path = os.path.join(run_dir, log_file_name)
        if not os.path.isfile(log_file_path):
            print(f"Log file {log_file_path} not found, skipping...")
            continue

        iterations, rewards = parse_log_file(log_file_path)
        all_run_rewards.append(rewards)

        for i, iteration in enumerate(iterations):
            if iteration not in all_iterations:
                all_iterations[iteration] = []
            all_iterations[iteration].append(rewards[i])
    
    # Sort the iterations
    sorted_iterations = sorted(all_iterations.keys())
    min_rewards = []
    max_rewards = []
    mean_rewards = []
    all_rewards_per_iteration = []

    for iteration in sorted_iterations:
        rewards = all_iterations[iteration]
        min_rewards.append(min(rewards))
        max_rewards.append(max(rewards))
        mean_rewards.append(np.mean(rewards))
        all_rewards_per_iteration.append(rewards)
    
    # Determine the best model
    best_iteration = sorted_iterations[np.argmax(mean_rewards)]
    
    return sorted_iterations, all_rewards_per_iteration, min_rewards, max_rewards, mean_rewards, best_iteration

def calculate_moving_average(data, window_size):
    moving_average = []
    for i in range(1, len(data) + 1):
        current_window_size = min(i, window_size)  # Gradually increase the window size
        window_data = data[i - current_window_size:i]  # Select the data in the current window
        window_average = np.mean(window_data)  # Compute the average for this window
        moving_average.append(window_average)
    return moving_average

def save_results_to_txt(iterations, all_rewards_per_iteration, min_rewards, max_rewards, mean_rewards, output_file_path, window_size):
    moving_average = calculate_moving_average(mean_rewards, window_size)
    
    with open(output_file_path, 'w') as f:
        # Write the header
        header = ["Iteration"]
        for run_idx in range(len(all_rewards_per_iteration[0])):
            header.append(f"Run_{run_idx+1} Mean Reward")
        header.extend(["Min Reward", "Max Reward", "Mean Reward", "Moving Average"])
        f.write("\t".join(header) + "\n")

        # Write the data
        for i, iteration in enumerate(iterations):
            line = [str(iteration)]
            line.extend([f"{reward:.6f}" for reward in all_rewards_per_iteration[i]])
            line.append(f"{min_rewards[i]:.6f}")
            line.append(f"{max_rewards[i]:.6f}")
            line.append(f"{mean_rewards[i]:.6f}")
            if i < len(moving_average):
                line.append(f"{moving_average[i]:.6f}")
            else:
                line.append("")  # In case moving_average is shorter
            f.write("\t".join(line) + "\n")

    print(f"Results saved to {output_file_path}")

def plot_results(iterations, min_rewards, max_rewards, mean_rewards, title, plot_path, window_size, mean_time, total_mean, ppo_parameters, studied_parameter, experiment_type):
    print("="*50)
    print(studied_parameter, experiment_type)
    print(mean_time)

    # Baseline values
    baseline_total_mean = 11.71 if experiment_type == "rough" else 15.39
    # baseline_best_reward = 10.21 if experiment_type == "rough" else 18.18

    # Calculate percentage differences
    total_mean_diff = ((total_mean - baseline_total_mean) / baseline_total_mean) * 100
    best_index = np.argmax(mean_rewards)
    best_iteration = iterations[best_index]
    best_reward = mean_rewards[best_index]
    # best_reward_diff = ((best_reward - baseline_best_reward) / baseline_best_reward) * 100

    # Print information to the terminal
    # print(f"Experiment: {title}")
    # print(f"Total Mean: {total_mean:.2f} ({total_mean_diff:+.2f}%) compared to baseline {baseline_total_mean}")
    # print(f"Best Reward: {best_reward:.2f} at iteration {best_iteration} ({best_reward_diff:+.2f}%) compared to baseline {baseline_best_reward}")
    # print("="*50)

    plt.figure(figsize=(10, 7))
    plt.grid(True, zorder=0, alpha=0.5)  # Draw grid behind the plot
    plt.fill_between(iterations, min_rewards, max_rewards, color='lightblue', alpha=0.7, zorder=1, label='Reward Range')
    plt.plot(iterations, mean_rewards, color='tab:blue', zorder=1, label='Episode Reward (mean)')

    # Calculate and plot the moving average
    if len(mean_rewards) >= window_size:
        moving_average = calculate_moving_average(mean_rewards, window_size)
        plt.plot(iterations, moving_average, color='black', linewidth=1.5, alpha=0.8, zorder=3, label='Moving Average Ep. Reward')

    # Plot a dashed red line for the total mean value with percentage difference
    plt.axhline(y=total_mean, color='red', linestyle='--', linewidth=2, alpha=0.5, label=f'Total Mean: {total_mean:.2f})', zorder=4) #({total_mean_diff:+.2f}\%)', zorder=4)

    # Plot a red dot on the best reward point with percentage difference
    plt.scatter(best_iteration, best_reward, color='red', s=20, alpha=0.5, zorder=5, label=f'Best Reward: {best_reward:.2f} (It. {best_iteration})')

    plt.xlim(0, len(iterations))
    if experiment_type == 'rough':
        plt.ylim(0, 19)
    elif experiment_type == 'obstacles':
        plt.ylim(0, 25)
    plt.xlabel('Episodes')
    plt.ylabel('Mean Reward')
    title = title + " - " + f"Mean Training Time: {mean_time/60:.2f} min"
    plt.suptitle(rf'\textbf{{{title}}}', fontsize=16, fontweight='bold')
        
    ppo_text = (
        (r"$\boldsymbol{n_{\text{\textbf{epochs}}}}$" if "num_learning_epochs" == studied_parameter else r"$n_{\text{epochs}}$") + r"= " + 
        (r"$\boldsymbol{" + ppo_parameters[3].split(": ")[1] + r"}$" if "num_learning_epochs" == studied_parameter else ppo_parameters[3].split(": ")[1]) + r" | " +
        
        (r"$\boldsymbol{n_{\text{\textbf{batches}}}}$" if "num_mini_batches" == studied_parameter else r"$n_{\text{batches}}$") + r"= " + 
        (r"$\boldsymbol{" + ppo_parameters[4].split(": ")[1] + r"}$" if "num_mini_batches" == studied_parameter else ppo_parameters[4].split(": ")[1]) + r" | " +
        
        (r"$\boldsymbol{\epsilon}$" if "clip_param" == studied_parameter else r"$\epsilon$") + r"= " + 
        (r"$\boldsymbol{" + ppo_parameters[2].split(": ")[1] + r"}$" if "clip_param" == studied_parameter else ppo_parameters[2].split(": ")[1]) + r" | " +
        
        (r"$\boldsymbol{\gamma}$" if "gamma" == studied_parameter else r"$\boldsymbol{\gamma}$") + r"= " +
        (r"$\boldsymbol{" + ppo_parameters[7].split(": ")[1] + r"}$" if "gamma" == studied_parameter else r"$\boldsymbol{" + ppo_parameters[7].split(": ")[1] + r"}$") + r" | " +
        
        (r"$\boldsymbol{\lambda}$" if "lam" == studied_parameter else r"$\lambda$") + r"= " + 
        (r"$\boldsymbol{" + ppo_parameters[8].split(": ")[1] + r"}$" if "lam" == studied_parameter else ppo_parameters[8].split(": ")[1]) + r"\\ " +
        
        (r"$\boldsymbol{c_{\text{\text{value}}}}$" if "value_loss_coef" == studied_parameter else r"$c_{\text{value}}$") + r"= " +
        (r"$\boldsymbol{" + ppo_parameters[5].split(": ")[1] + r"}$" if "value_loss_coef" == studied_parameter else ppo_parameters[5].split(": ")[1]) + r" | " +
        
        (r"$\boldsymbol{c_{\text{\textbf{entropy}}}}$" if "entropy_coef" == studied_parameter else r"$\boldsymbol{c_{\text{\textbf{entropy}}}}$") + r"= " +
        (r"$\boldsymbol{" + ppo_parameters[6].split(": ")[1] + r"}$" if "entropy_coef" == studied_parameter else r"$\boldsymbol{" + ppo_parameters[6].split(": ")[1]) + r"}$" + r" | " +
        
        (r"$\boldsymbol{\alpha}$" if "learning_rate" == studied_parameter else r"$\alpha$") + r"= " +
        (r"$\boldsymbol{" + ppo_parameters[1].split(": ")[1] + r"}$" if "learning_rate" == studied_parameter else ppo_parameters[1].split(": ")[1]) + r" | " +
        
        (r"$\boldsymbol{\|g\|_{\text{\textbf{max}}}}$" if "max_grad_norm" == studied_parameter else r"$\|g\|_{\text{max}}$") + r"= " +
        (r"$\boldsymbol{" + ppo_parameters[9].split(": ")[1] + r"}$" if "max_grad_norm" == studied_parameter else ppo_parameters[9].split(": ")[1]) + r" | " +

        (r"$\boldsymbol{\text{\textbf{KL}}_{\text{\textbf{desired}}}}$" if "desired_kl" == studied_parameter else r"$\text{KL}_{\text{desired}}$") + r"= " +
        (r"$\boldsymbol{" + ppo_parameters[0].split(": ")[1] + r"}$" if "desired_kl" == studied_parameter else ppo_parameters[0].split(": ")[1]) 
    )

    subtitle = (f"{ppo_text}") 

    # Center the subtitle text
    plt.figtext(0.5, 0.92, subtitle, wrap=True, ha='center', fontsize=14)

    plt.legend(loc='upper left')
    
    # Save the figure
    plt.savefig(plot_path.replace('.png', '.png'), bbox_inches='tight', transparent=True, format='png', dpi=300)
    plt.close()
    print(f"Figure saved as {plot_path.replace('.png', '.png')}")



def process_logs_in_directory(directory_path, experiment_type):
    if experiment_type == 'rough':
        log_file_name = 'rough_train.log'
    elif experiment_type == 'obstacles':
        log_file_name = 'obs_train.log'
    else:
        print(f"Unknown experiment type: {experiment_type}, skipping...")
        return
    
    experiment_path = os.path.join(directory_path, experiment_type)
    if not os.path.isdir(experiment_path):
        print(f"Experiment directory {experiment_path} not found, skipping...")
        return
    
    run_directories = glob.glob(os.path.join(experiment_path, 'run_*'))
    
    if not run_directories:
        print(f"No run directories found in {experiment_path}, skipping...")
        return
    
    # Extract the studied parameter from the folder name
    studied_parameter = get_studied_parameter_from_folder_name(directory_path)

    # Extract PPO parameters from the rough_train.log file within the first run_* directory
    first_run_dir = run_directories[0]  # Take the first run_* directory
    ppo_parameters = extract_ppo_parameters(os.path.join(first_run_dir, log_file_name))

    # Process all run directories
    iterations, all_rewards_per_iteration, min_rewards, max_rewards, mean_rewards, _ = process_run_directories(run_directories, log_file_name)
    
    if not iterations:
        print(f"No valid data found in {experiment_path}, skipping...")
        return
    
    # Extract training times from the experiment.log
    experiment_log_file_path = os.path.join(directory_path, 'experiment.log')
    if not os.path.isfile(experiment_log_file_path):
        print(f"experiment.log file not found in {directory_path}, skipping...")
        return
    
    times, _ = parse_experiment_log(experiment_log_file_path)
    print("times:", times)
    if experiment_type == 'rough':
        mean_time = np.mean(times[:5]) if times[:5] else 0
    elif experiment_type == 'obstacles':
        mean_time = np.mean(times[5:]) if times[5:] else 0

    # Calculate total mean of the plotted values
    total_mean = np.mean(mean_rewards)

    # Determine title, plot path, and txt file path
    if experiment_type == 'rough':
        title = "General Purpose Locomotion - Stage 1"
        plot_file_name = "rough_plot.png"
        txt_file_name = "rough_results.txt"
    elif experiment_type == 'obstacles':
        title = "Obstacle Avoidance Locomotion - Stage 2"
        plot_file_name = "obstacles_plot.png"
        txt_file_name = "obstacles_results.txt"
    
    plot_path = os.path.join(directory_path, plot_file_name)
    txt_file_path = os.path.join(directory_path, txt_file_name)
    
    # Generate the plot
    window_size = 20  # moving average window size
    plot_results(iterations, min_rewards, max_rewards, mean_rewards, title, plot_path, window_size, mean_time, total_mean, ppo_parameters, studied_parameter, experiment_type)
    
    # Save the results to a .txt file
    save_results_to_txt(iterations, all_rewards_per_iteration, min_rewards, max_rewards, mean_rewards, txt_file_path, window_size)


def traverse_directories(parent_directory):
    # Use glob to find all subdirectories
    subdirectories = glob.glob(os.path.join(parent_directory, '*/'))

    # Iterate through each subdirectory
    for subdir in subdirectories:
        # Skip hidden directories (those starting with a dot)
        if os.path.basename(subdir).startswith('.'):
            print(f"Skipping hidden directory: {subdir}")
            continue

        # Process the logs in the 'rough' subdirectory
        process_logs_in_directory(subdir, 'rough')

        # Process the logs in the 'obstacles' subdirectory
        process_logs_in_directory(subdir, 'obstacles')

# Example usage:
parent_folder = '.'  # Change this to the path of the parent folder
traverse_directories(parent_folder)


Experiment directory ./my_next_experiments/rough not found, skipping...
Experiment directory ./my_next_experiments/obstacles not found, skipping...
./02092024_002149_new_baseline/
<re.Match object; span=(4, 34), match='Training completed in 01:06:13'>
<re.Match object; span=(4, 34), match='Training completed in 01:05:51'>
<re.Match object; span=(4, 34), match='Training completed in 01:05:59'>
<re.Match object; span=(4, 34), match='Training completed in 01:06:20'>
<re.Match object; span=(4, 34), match='Training completed in 01:06:08'>
<re.Match object; span=(4, 34), match='Training completed in 00:15:08'>
<re.Match object; span=(4, 34), match='Training completed in 00:15:12'>
<re.Match object; span=(4, 34), match='Training completed in 00:15:01'>
<re.Match object; span=(4, 34), match='Training completed in 00:14:55'>
<re.Match object; span=(4, 34), match='Training completed in 00:15:16'>
times: [3973, 3951, 3959, 3980, 3968, 908, 912, 901, 895, 916]
None rough
3966.2
Figure saved as ./0

In [22]:
import os
import shutil
import re

# Define the current directory and the target directory
current_dir = os.getcwd()
parent_dir = "."
results_dir = os.path.join(parent_dir, 'results')

# Create the 'results' directory in the parent directory if it doesn't exist
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Function to extract the meaningful part of the folder name
def extract_name(folder_name):
    # Assuming the folder name is structured as date_time_otherstuff
    # We want to keep only the "otherstuff" part
    parts = folder_name.split('_', 2)  # Split only the first two underscores
    if len(parts) == 3:
        return parts[2]
    return folder_name  # If the structure doesn't match, return the original name

# Iterate over the folders in the current directory
for folder in os.listdir(current_dir):
    folder_path = os.path.join(current_dir, folder)
    if os.path.isdir(folder_path):  # Check if it's a directory
        # Extract the meaningful part of the folder name
        clean_name = extract_name(folder)
        
        # Define paths for the files to be copied
        obstacles_file = os.path.join(folder_path, 'obstacles_results.txt')
        rough_file = os.path.join(folder_path, 'rough_results.txt')
        
        # Copy and rename the files if they exist
        if os.path.exists(obstacles_file):
            new_obstacles_file = os.path.join(results_dir, f"{clean_name}_obstacles.txt")
            shutil.copy(obstacles_file, new_obstacles_file)
        
        if os.path.exists(rough_file):
            new_rough_file = os.path.join(results_dir, f"{clean_name}_rough.txt")
            shutil.copy(rough_file, new_rough_file)

print(f"Files have been copied and renamed successfully to {results_dir}.")

Files have been copied and renamed successfully to ./results.


In [44]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import matplotlib as mpl

# LaTeX style setup
pgf_with_latex = {
    'pgf.texsystem': 'pdflatex',
    'text.usetex': True,
    'font.size': 10,
    'font.family': 'serif',
    'font.serif': [],
    'font.sans-serif': [],
    'font.monospace': [],
    'text.latex.preamble': r'\usepackage[utf8]{inputenc}\usepackage[T1]{fontenc}\usepackage[detect-all]{siunitx}\usepackage{bm}\usepackage{amsmath}\usepackage{color}\usepackage{amsbsy}'
}

mpl.rcParams.update(pgf_with_latex)
plt.rcParams["axes.axisbelow"] = True  # Ensure that the grid is drawn behind everything else

experiment_name_mapping = {
    "clip_param": r"$\epsilon$",
    "lam": r"$\lambda$",
    "gamma": r"$\gamma$",
    "learning_rate": r"$\alpha$",
    "max_grad_norm": r"$\|g\|_{\text{max}}$",
    "num_learning_epochs": r"$n_{\text{epochs}}$",
    "num_mini_batches": r"$n_{\text{batches}}$",
    "value_loss_coef": r"$c_{\text{value}}$",
    "entropy_coef": r"$c_{\text{entropy}}$",
    "desired_kl": r"$\text{KL}_{\text{desired}}$"
}

baseline_values = {
    "clip_param": "0.2",
    "lam": "0.95",
    "gamma": "0.99",
    "learning_rate": "1e-3",
    "max_grad_norm": "1.0",
    "num_learning_epochs": "5",
    "num_mini_batches": "4",
    "value_loss_coef": "1.0",
    "entropy_coef": "0.01",
    "desired_kl": "0.01",
    "seed": "1"
}

def extract_experiment_names(results_dir):
    """Extract unique experiment categories from the files in the results directory."""
    experiment_names = set()
    
    for file_name in os.listdir(results_dir):
        print(f"Checking file: {file_name}")  # Debugging line
        match = re.match(r'([a-zA-Z_]+\d*\.*\d*)_([a-zA-Z_]+\d*\.*\d*)_(rough|obstacles)\.txt', file_name)
        if match:
            experiment_name = f"{match.group(1)}_{match.group(2)}"
            experiment_names.add(experiment_name)
            print(f"Found experiment: {experiment_name}")  # Debugging line
    
    return sorted(experiment_names)

def find_experiment_files(results_dir, experiment_name):
    """Find and return files for a given experiment category."""
    rough_files = []
    obstacles_files = []
    
    for file_name in os.listdir(results_dir):
        if file_name.startswith(experiment_name) and file_name.endswith('_rough.txt'):
            rough_files.append(os.path.join(results_dir, file_name))
            print(f"Found rough file: {file_name}")  # Debugging line
        elif file_name.startswith(experiment_name) and file_name.endswith('_obstacles.txt'):
            obstacles_files.append(os.path.join(results_dir, file_name))
            print(f"Found obstacles file: {file_name}")  # Debugging line
    
    if rough_files and obstacles_files:
        return rough_files, obstacles_files
    else:
        print(f"Missing files for experiment: {experiment_name}")  # Debugging line
        return None

def get_experiment_title(experiment_name):
    components = experiment_name.split('_')
    title_parts = []
    for component in components:
        for key in experiment_name_mapping:
            if key in component:
                title_parts.append(component.replace(key, experiment_name_mapping[key]))
                break
        else:
            title_parts.append(component)
    
    return " ".join(title_parts).title()

def plot_experiment(files_rough, files_obstacles, baseline_file_rough, baseline_file_obstacles, experiment_name):
    print(50*"=")
    print(files_rough, files_obstacles, baseline_file_rough, baseline_file_obstacles, experiment_name)
    fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharey=True)  # Create two subplots side by side

    colors = plt.cm.tab10.colors  # Use the tab10 colormap, which provides 10 distinct colors
    value_to_color = {}  # Dictionary to map hyperparameter values to colors

    # Plotting for rough
    axs[0].set_title('Rough')
    plot_single_experiment(axs[0], files_rough, baseline_file_rough, experiment_name, colors, value_to_color)

    # Plotting for obstacles
    axs[1].set_title('Obstacles')
    plot_single_experiment(axs[1], files_obstacles, baseline_file_obstacles, experiment_name, colors, value_to_color)
    
    title = get_experiment_title(experiment_name)
    fig.suptitle(f'Baseline vs. New Params', fontsize=16)

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to include suptitle
    
    output_dir = '/home/josep-barbera/Documents/nViNL/experiments/results'  # Adjust this path as needed
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f'{experiment_name}_comparison.png')
    plt.savefig(output_path, dpi=200, transparent=True)
    plt.close(fig)  # Close the figure to free up memory
    print(f"Figure saved as {output_path}")  # Debugging line

def plot_single_experiment(ax, files, baseline_file, experiment_name, colors, value_to_color):
    """Helper function to plot a single experiment on a given axis."""
    
    # Extract parameter names and values from experiment_name
    match = re.match(r'([a-zA-Z_]+)(\d*\.*\d*)_([a-zA-Z_]+)(\d*\.*\d*)', experiment_name)
    if match:
        param1, value1, param2, value2 = match.groups()
        
        # Clean up the parameter names (remove any trailing underscores)
        param1 = param1.rstrip('_')
        param2 = param2.rstrip('_')
        
        # Retrieve LaTeX symbols for both parameters
        param1_label = experiment_name_mapping.get(param1, param1)
        param2_label = experiment_name_mapping.get(param2, param2)
        
        # Get the correct baseline values for both parameters
        baseline_value1 = baseline_values.get(param1, "N/A")
        baseline_value2 = baseline_values.get(param2, "N/A")
        
        # Format the baseline label to include both parameters
        baseline_label = f"baseline: {param1_label} = {baseline_value1}, {param2_label} = {baseline_value2}"

        # Plot baseline first
        data = pd.read_csv(baseline_file, sep='\t')
        moving_average = data['Moving Average']
        min_reward = data['Min Reward']
        max_reward = data['Max Reward']

        ax.fill_between(data.index, min_reward, max_reward, color='gray', alpha=0.3)
        ax.plot(data.index, moving_average, color='gray', label=baseline_label, linestyle='--')

        # Extract the values and ensure consistent order
        file_paths_sorted = sorted(files, key=lambda x: os.path.basename(x).split('_')[-2])
        labels_and_colors = {}

        # Assign colors consistently
        for i, file_path in enumerate(file_paths_sorted):
            data = pd.read_csv(file_path, sep='\t')
            moving_average = data['Moving Average']
            min_reward = data['Min Reward']
            max_reward = data['Max Reward']

            # Generate label with both parameters and their values
            label = f"New params: {param1_label} = {value1}, {param2_label} = {value2}"

            # Ensure consistent color assignment
            color = colors[i % len(colors)]

            labels_and_colors[label] = color

            ax.fill_between(data.index, min_reward, max_reward, color=color, alpha=0.3)
            ax.plot(data.index, moving_average, color=color, label=label)

        # Retrieve and sort the legend handles by our custom ordering
        handles, labels = ax.get_legend_handles_labels()
        sorted_labels_and_handles = sorted(
            zip(labels, handles),
            key=lambda t: (0 if "baseline" in t[0] else (1 if labels_and_colors[t[0]] == colors[0] else 2))
        )
        sorted_labels, sorted_handles = zip(*sorted_labels_and_handles)

        ax.legend(sorted_handles, sorted_labels, loc='upper left')
        ax.set_xlim(0, len(data.index))
        ax.set_xlabel('Episodes')
        ax.set_ylabel('Mean Reward')
        ax.grid(True, zorder=0, alpha=0.5)

    else:
        print(f"Could not match the experiment name pattern: {experiment_name}")



def main():
    print("Current directory:", os.getcwd())
    results_dir = './results'
    
    experiment_names = extract_experiment_names(results_dir)
    print("Experiment names found:", experiment_names)

    for experiment in experiment_names:
        result = find_experiment_files(results_dir, experiment)
        print(result)
        
        if result:
            rough_files, obstacles_files = result
            baseline_rough = os.path.join(results_dir, 'baseline_rough.txt')
            print(baseline_rough)
            baseline_obstacles = os.path.join(results_dir, 'baseline_obstacles.txt')
            print(experiment)
            plot_experiment(rough_files, obstacles_files, baseline_rough, baseline_obstacles, experiment)
        else:
            print(f"Skipping experiment {experiment} - incomplete data (missing required files).")

if __name__ == "__main__":
    main()


Current directory: /home/josep-barbera/Documents/nViNL/experiments
Checking file: clip_param_0.3_obstacles.txt
Found experiment: clip_param_0.3
Checking file: clip_param_0.3_rough.txt
Found experiment: clip_param_0.3
Checking file: clip_param_0.1_rough.txt
Found experiment: clip_param_0.1
Checking file: baseline_obstacles.txt
Checking file: gamma_0.95_entropy_coef_0.005_comparison.png
Checking file: gamma_0.95_entropy_coef_0.005_obstacles.txt
Found experiment: gamma_0.95_entropy_coef_0.005
Checking file: gamma_0.95_entropy_coef_0.005_rough.txt
Found experiment: gamma_0.95_entropy_coef_0.005
Checking file: clip_param_0.1_obstacles.txt
Found experiment: clip_param_0.1
Checking file: baseline_rough.txt
Checking file: clip_param_0.3_comparison.png
Checking file: done
Checking file: clip_param_0.1_comparison.png
Experiment names found: ['clip_param_0.1', 'clip_param_0.3', 'gamma_0.95_entropy_coef_0.005']
Found rough file: clip_param_0.1_rough.txt
Found obstacles file: clip_param_0.1_obstacl

In [107]:
import os
import re
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages

# LaTeX style setup
pgf_with_latex = {
    'pgf.texsystem': 'pdflatex',
    'text.usetex': True,
    'font.size': 10,
    'font.family': 'serif',
    'font.serif': [],
    'font.sans-serif': [],
    'font.monospace': [],
    'text.latex.preamble': r'\usepackage[utf8]{inputenc}\usepackage[T1]{fontenc}\usepackage[detect-all]{siunitx}\usepackage{bm}\usepackage{amsmath}\usepackage{color}\usepackage{amsbsy}'
}

mpl.rcParams.update(pgf_with_latex)
plt.rcParams["axes.axisbelow"] = True  # Ensure that the grid is drawn behind everything else

def parse_log_file(file_path):
    iterations = []
    rewards = []
    with open(file_path, 'r') as file:
        for line in file:
            iteration_match = re.search(r'Learning iteration (\d+)/', line)
            if iteration_match:
                iteration = int(iteration_match.group(1))
            
            reward_match = re.search(r'Mean reward: ([\d\.-]+)', line)
            if reward_match:
                reward = float(reward_match.group(1))
                iterations.append(iteration)
                rewards.append(reward)
    return iterations, rewards

def get_studied_parameter_from_folder_name(folder_name):
    match = re.search(r'^\./\d{8}_\d{6}_(\w+)_\d*\.?\d+/?$', folder_name)
    if match:
        return match.group(1)  # return the parameter name
    else:
        return None

def extract_ppo_parameters(file_path):
    parameters = []
    inside_ppo_section = False
    with open(file_path, 'r') as file:
        for line in file:
            if "##### PPO PARAMETERS ######" in line:
                inside_ppo_section = True
            elif "################################################################################" in line:
                if inside_ppo_section:
                    break  # Exit after the PPO section ends
            elif inside_ppo_section:
                param_match = re.search(r'^\s*(\w+):\s+([\d\.\-e]+)\s*$', line)
                if param_match:
                    parameters.append(f"{param_match.group(1)}: {param_match.group(2)}")
    return parameters

def process_run_directories(run_directories, log_file_name):
    all_iterations = {}
    all_run_rewards = []

    for run_dir in run_directories:
        log_file_path = os.path.join(run_dir, log_file_name)
        if not os.path.isfile(log_file_path):
            print(f"Log file {log_file_path} not found, skipping...")
            continue

        iterations, rewards = parse_log_file(log_file_path)
        all_run_rewards.append(rewards)

        for i, iteration in enumerate(iterations):
            if iteration not in all_iterations:
                all_iterations[iteration] = []
            all_iterations[iteration].append(rewards[i])
    
    # Sort the iterations
    sorted_iterations = sorted(all_iterations.keys())
    min_rewards = []
    max_rewards = []
    mean_rewards = []
    all_rewards_per_iteration = []

    for iteration in sorted_iterations:
        rewards = all_iterations[iteration]
        min_rewards.append(min(rewards))
        max_rewards.append(max(rewards))
        mean_rewards.append(np.mean(rewards))
        all_rewards_per_iteration.append(rewards)
    
    # Determine the best model
    best_iteration = sorted_iterations[np.argmax(mean_rewards)]
    
    return sorted_iterations, all_rewards_per_iteration, min_rewards, max_rewards, mean_rewards, best_iteration

def calculate_moving_average(data, window_size):
    moving_average = []
    for i in range(1, len(data) + 1):
        current_window_size = min(i, window_size)  # Gradually increase the window size
        window_data = data[i - current_window_size:i]  # Select the data in the current window
        window_average = np.mean(window_data)  # Compute the average for this window
        moving_average.append(window_average)
    return moving_average

def save_results_to_txt(iterations, all_rewards_per_iteration, min_rewards, max_rewards, mean_rewards, output_file_path, window_size):
    moving_average = calculate_moving_average(mean_rewards, window_size)
    
    with open(output_file_path, 'w') as f:
        header = ["Iteration"]
        for run_idx in range(len(all_rewards_per_iteration[0])):
            header.append(f"Run_{run_idx+1} Mean Reward")
        header.extend(["Min Reward", "Max Reward", "Mean Reward", "Moving Average"])
        f.write("\t".join(header) + "\n")

        for i, iteration in enumerate(iterations):
            line = [str(iteration)]
            line.extend([f"{reward:.6f}" for reward in all_rewards_per_iteration[i]])
            line.append(f"{min_rewards[i]:.6f}")
            line.append(f"{max_rewards[i]:.6f}")
            line.append(f"{mean_rewards[i]:.6f}")
            if i < len(moving_average):
                line.append(f"{moving_average[i]:.6f}")
            else:
                line.append("")  # In case moving_average is shorter
            f.write("\t".join(line) + "\n")

    print(f"Results saved to {output_file_path}")

def plot_results(iterations, min_rewards, max_rewards, mean_rewards, title, plot_path, window_size, mean_time, total_mean, ppo_parameters, studied_parameter, experiment_type):
    if studied_parameter == 'desired_kl' and ppo_parameters[0].split(": ")[1] == "0.005" and experiment_type == "obstacles":
        print("hellllooooo")
        iterations = [x - 1000 for x in iterations]
    plt.figure(figsize=(10, 7))
    plt.grid(True, zorder=0, alpha=0.5)  # Draw grid behind the plot
    # plt.fill_between(iterations, min_rewards, max_rewards, color='lightblue', alpha=0.7, zorder=1, label='Reward Range')
    plt.plot(iterations, mean_rewards, color='tab:blue', zorder=1, label='Episode Reward (mean)')

    if len(mean_rewards) >= window_size:
        moving_average = calculate_moving_average(mean_rewards, window_size)
        plt.plot(iterations, moving_average, color='black', linewidth=1.5, alpha=0.8, zorder=3, label='Moving Average Ep. Reward')

    best_index = np.argmax(mean_rewards)
    best_iteration = iterations[best_index]
    best_reward = mean_rewards[best_index]

    plt.axhline(y=total_mean, color='red', linestyle='--', linewidth=2, alpha=0.5, label=f'Total Mean: {total_mean:.2f}', zorder=4)

    plt.scatter(best_iteration, best_reward, color='red', s=20, alpha=0.5, zorder=5, label=f'Best Reward: {best_reward:.2f} (It. {best_iteration})')
    
    plt.xlim(0, len(iterations))
    if experiment_type == 'rough':
        plt.ylim(0, 17)
    elif experiment_type == 'obstacles':
        plt.ylim(0, 21)

    plt.xlabel('Episodes')
    plt.ylabel('Mean Reward')
    # title = title + " - " + f"Mean Training Time: {mean_time/60:.2f} min"
    plt.title(rf'\textbf{{{title}}}', fontsize=16, fontweight='bold')
        
    # ppo_text = (
    #     (r"$\boldsymbol{n_{\text{\textbf{epochs}}}}$" if "num_learning_epochs" == studied_parameter else r"$n_{\text{epochs}}$") + r"= " + 
    #     (r"$\boldsymbol{" + ppo_parameters[3].split(": ")[1] + r"}$" if "num_learning_epochs" == studied_parameter else ppo_parameters[3].split(": ")[1]) + r" | " +
        
    #     (r"$\boldsymbol{n_{\text{\textbf{batches}}}}$" if "num_mini_batches" == studied_parameter else r"$n_{\text{batches}}$") + r"= " + 
    #     (r"$\boldsymbol{" + ppo_parameters[4].split(": ")[1] + r"}$" if "num_mini_batches" == studied_parameter else ppo_parameters[4].split(": ")[1]) + r" | " +
        
    #     (r"$\boldsymbol{\epsilon}$" if "clip_param" == studied_parameter else r"$\epsilon$") + r"= " + 
    #     (r"$\boldsymbol{" + ppo_parameters[2].split(": ")[1] + r"}$" if "clip_param" == studied_parameter else ppo_parameters[2].split(": ")[1]) + r" | " +
        
    #     (r"$\boldsymbol{\gamma}$" if "gamma" == studied_parameter else r"$\gamma$") + r"= " +
    #     (r"$\boldsymbol{" + ppo_parameters[7].split(": ")[1] + r"}$" if "gamma" == studied_parameter else ppo_parameters[7].split(": ")[1]) + r" | " +
        
    #     (r"$\boldsymbol{\lambda}$" if "lam" == studied_parameter else r"$\lambda$") + r"= " + 
    #     (r"$\boldsymbol{" + ppo_parameters[8].split(": ")[1] + r"}$" if "lam" == studied_parameter else ppo_parameters[8].split(": ")[1]) + r"\\ " +
        
    #     (r"$\boldsymbol{c_{\text{\text{value}}}}$" if "value_loss_coef" == studied_parameter else r"$c_{\text{value}}$") + r"= " +
    #     (r"$\boldsymbol{" + ppo_parameters[5].split(": ")[1] + r"}$" if "value_loss_coef" == studied_parameter else ppo_parameters[5].split(": ")[1]) + r" | " +
        
    #     (r"$\boldsymbol{c_{\text{\textbf{entropy}}}}$" if "entropy_coef" == studied_parameter else r"$c_{\text{entropy}}$") + r"= " +
    #     (r"$\boldsymbol{" + ppo_parameters[6].split(": ")[1] + r"}$" if "entropy_coef" == studied_parameter else ppo_parameters[6].split(": ")[1]) + r" | " +
        
    #     (r"$\boldsymbol{\alpha}$" if "learning_rate" == studied_parameter else r"$\alpha$") + r"= " +
    #     (r"$\boldsymbol{" + ppo_parameters[1].split(": ")[1] + r"}$" if "learning_rate" == studied_parameter else ppo_parameters[1].split(": ")[1]) + r" | " +
        
    #     (r"$\boldsymbol{\|g\|_{\text{\textbf{max}}}}$" if "max_grad_norm" == studied_parameter else r"$\|g\|_{\text{max}}$") + r"= " +
    #     (r"$\boldsymbol{" + ppo_parameters[9].split(": ")[1] + r"}$" if "max_grad_norm" == studied_parameter else ppo_parameters[9].split(": ")[1]) + r" | " +

    #     (r"$\boldsymbol{\text{\textbf{KL}}_{\text{\textbf{desired}}}}$" if "desired_kl" == studied_parameter else r"$\text{KL}_{\text{desired}}$") + r"= " +
    #     (r"$\boldsymbol{" + ppo_parameters[0].split(": ")[1] + r"}$" if "desired_kl" == studied_parameter else ppo_parameters[0].split(": ")[1]) 
    # )

    # subtitle = (f"{ppo_text}")

    # plt.figtext(0.5, 0.92, subtitle, wrap=True, ha='center', fontsize=14)

    plt.legend(loc='lower left')

    plt.savefig(plot_path.replace('.png', '.png'), bbox_inches='tight', transparent = True, format='png', dpi=300)
    plt.close()
    print(f"Figure saved as {plot_path.replace('.png', '.png')}")

def process_logs_in_directory(directory_path, experiment_type):
    if experiment_type == 'rough':
        log_file_name = 'rough_train.log'
    elif experiment_type == 'obstacles':
        log_file_name = 'obs_train.log'
    else:
        print(f"Unknown experiment type: {experiment_type}, skipping...")
        return
    
    experiment_path = os.path.join(directory_path, experiment_type)
    if not os.path.isdir(experiment_path):
        print(f"Experiment directory {experiment_path} not found, skipping...")
        return
    
    run_directories = glob.glob(os.path.join(experiment_path, 'run_*'))
    
    if not run_directories:
        print(f"No run directories found in {experiment_path}, skipping...")
        return
    
    studied_parameter = get_studied_parameter_from_folder_name(directory_path)

    first_run_dir = run_directories[0]
    ppo_parameters = extract_ppo_parameters(os.path.join(first_run_dir, log_file_name))

    iterations, all_rewards_per_iteration, min_rewards, max_rewards, mean_rewards, _ = process_run_directories(run_directories, log_file_name)
    
    if not iterations:
        print(f"No valid data found in {experiment_path}, skipping...")
        return
    
    experiment_log_file_path = os.path.join(directory_path, 'experiment.log')
    mean_time = 0  # Default value for mean time

    if os.path.isfile(experiment_log_file_path):
        times, _ = parse_experiment_log(experiment_log_file_path)
        mean_time = np.mean(times) if times else 0

    total_mean = np.mean(mean_rewards)

    if experiment_type == 'rough':
        title = "General Purpose Locomotion - Stage 1"
        plot_file_name = "rough_plot.png"
        txt_file_name = "rough_results.txt"
    elif experiment_type == 'obstacles':
        title = "Obstacle Avoidance Locomotion - Stage 2"
        plot_file_name = "obstacles_plot.png"
        txt_file_name = "obstacles_results.txt"
    
    plot_path = os.path.join(directory_path, plot_file_name)
    txt_file_path = os.path.join(directory_path, txt_file_name)
    
    window_size = 20  # moving average window size
    plot_results(iterations, min_rewards, max_rewards, mean_rewards, title, plot_path, window_size, mean_time, total_mean, ppo_parameters, studied_parameter, experiment_type)
    
def traverse_directories(parent_directory):
    subdirectories = glob.glob(os.path.join(parent_directory, '*/'))

    for subdir in subdirectories:
        if os.path.basename(subdir).startswith('.'):
            print(f"Skipping hidden directory: {subdir}")
            continue

        process_logs_in_directory(subdir, 'rough')
        process_logs_in_directory(subdir, 'obstacles')

# Example usage:
parent_folder = './old/'  # Change this to the path of the parent folder
traverse_directories(parent_folder)


Figure saved as ./old/example_experiment_20240823_063325/rough_plot.png
Figure saved as ./old/example_experiment_20240823_063325/obstacles_plot.png
Figure saved as ./old/example_experiment_20240822_215620/rough_plot.png
Figure saved as ./old/example_experiment_20240822_215620/obstacles_plot.png
Experiment directory ./old/example_experiment_20240630_104019/rough not found, skipping...
Experiment directory ./old/example_experiment_20240630_104019/obstacles not found, skipping...
Experiment directory ./old/example_experiment_20240629_163213/rough not found, skipping...
Experiment directory ./old/example_experiment_20240629_163213/obstacles not found, skipping...
Experiment directory ./old/example_experiment_20240821_151510/rough not found, skipping...
Experiment directory ./old/example_experiment_20240821_151510/obstacles not found, skipping...
Experiment directory ./old/example_experiment_20240821_214515/rough not found, skipping...
Experiment directory ./old/example_experiment_20240821_